# Реализация метода

## База

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

def mixed_norm_polyfit(x, y, c1, c2, degree):
    """
    Поиск коэффициентов полинома, минимизирующего линейную комбинацию L1 и L2 ошибок.
    
    Потери для одной точки: c1 * |F(x_i) - y_i| + c2 * (F(x_i) - y_i)^2
    Итоговые потери: сумма по всем точкам.
    
    Аргументы:
        x : array-like, форма (n_samples,)
            Входные значения
        y : array-like, форма (n_samples,)
            Целевые значения
        c1 : float
            Коэффициент для L1-члена (должен быть >= 0)
        c2 : float
            Коэффициент для L2-члена (должен быть >= 0)
        degree : int
            Степень полинома
    
    Возвращает:
        coefficients : ndarray, форма (degree + 1,)
            Коэффициенты полинома [a0, a1, ..., a_degree] для F(x) = a0 + a1*x + ... + a_degree*x^degree
        y_pred : ndarray, форма (n_samples,)
            Предсказанные значения на входных x
    """
    x = np.asarray(x, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    
    if x.shape[0] == 0 or y.shape[0] == 0:
        return np.zeros(degree + 1), np.array([])
    
    if x.shape[0] != y.shape[0]:
        raise ValueError("Длины x и y должны совпадать")
    
    if c1 < 0 or c2 < 0:
        raise ValueError("Коэффициенты c1 и c2 должны быть неотрицательными")
    
    if c2 == 0:
        c2 = 1e-15
    
    # Создаём матрицу Вандермонда: столбцы [x^0, x^1, ..., x^degree]
    X = np.vander(x, N=degree + 1, increasing=True)
    
    # Начальное приближение через МНК (устойчиво даже для недоопределённых систем)
    try:
        coef0, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    except np.linalg.LinAlgError:
        coef0 = np.zeros(degree + 1)
    
    # Вырожденный случай: нулевые веса
    if c1 == 0 and c2 == 0:
        return coef0, X @ coef0
    
    # Целевая функция с защитой от переполнения
    def loss(coef):
        residuals = X @ coef - y
        # Для устойчивости: избегаем возведения в квадрат очень больших чисел
        if c2 > 0 and np.any(np.abs(residuals) > 1e10):
            return np.inf
        l1_term = np.sum(np.abs(residuals))
        l2_term = np.sum(residuals ** 2) if c2 > 0 else 0.0
        return c1 * l1_term + c2 * l2_term
    
    # Основная оптимизация: метод Пауэлла (хорошо работает с негладкими функциями)
    result = minimize(
        loss,
        coef0,
        method='Powell',
        options={'maxiter': 10000, 'disp': False}
    )
    
    # Резервный метод при неудаче
    if not result.success:
        result = minimize(
            loss,
            coef0,
            method='Nelder-Mead',
            options={'maxiter': 10000, 'disp': False}
        )
    
    coefficients = result.x
    y_pred = X @ coefficients
    
    return coefficients, y_pred

In [9]:
x = np.array([0, 1, 2, 3, 4])
y = np.array([1, 3, 7, 13, 21])  # Соответствует y = x² + x + 1

# Подгонка квадратичного полинома с акцентом на устойчивость к выбросам (L1)
coef, y_pred = mixed_norm_polyfit(x, y, c1=1.0, c2=1, degree=2)
print("Коэффициенты:", coef)  # Ожидаемый результат: [~1, ~1, ~1]
print("Предсказания:", y_pred)

Коэффициенты: [1. 1. 1.]
Предсказания: [ 1.  3.  7. 13. 21.]


## Подбор c1 и с2 по MSE

In [10]:
import numpy as np

def select_best_mixed_norm(x, y, degree):
    # Сетка соотношений (a:b) → нормируем до суммы=1 для корректного сравнения
    ratios = [(1, 0), (0, 1), (1, 1), (3, 4), (4, 3), 
              (1, 2), (2, 1), (1, 4), (4, 1)]
    
    best_mse = np.inf
    best_coefficients = None
    best_y_pred = None
    best_c1 = best_c2 = None

    for a, b in ratios:
        total = a + b
        c1_val = a / total
        c2_val = b / total
        
        # Вызов существующей функции (внутри обрабатывает c2=0 → 1e-15)
        coefficients, y_pred = mixed_norm_polyfit(x, y, c1_val, c2_val, degree)
        
        # MSE на обучающих данных (как указано в ТЗ)
        mse = np.mean((y - y_pred) ** 2)
        
        if mse < best_mse:
            best_mse = mse
            best_coefficients = coefficients
            best_y_pred = y_pred
            best_c1 = c1_val
            best_c2 = c2_val
    
    return best_coefficients, best_y_pred, best_c1, best_c2

In [18]:
x = np.array([0, 1, 2, 3, 4])
y = np.array([1, 3, 7, 13, 21])  # Соответствует y = x² + x + 1

select_best_mixed_norm(x, y, degree=3)

(array([1., 1., 1., 0.]), array([ 1.,  3.,  7., 13., 21.]), 1.0, 0.0)

## Функция для подсчёта BIC

In [ ]:
import numpy as np
from scipy.integrate import nquad

def bic_entropy_stable(X, y, coef, degree, c1, c2, param_range=(-10, 10)):
    """
    Аргументы:
        X : ndarray (n, k) — матрица Вандермонда
        y : ndarray (n,)   — целевые значения
        coef : ndarray (k,) — оценённые коэффициенты
        degree : int        — степень полинома
        c1, c2 : float      — веса норм
        param_range : tuple или list[tuple] — границы интегрирования для каждого параметра
    
    Возвращает:
        float : значение критерия BIC
    """
    n, k = len(y), degree + 1
    ranges = [param_range] * k if isinstance(param_range, tuple) else param_range
    
    # 1. Вычисляем N_min в точке оценки (гарантированный минимум функции потерь)
    residuals_opt = X @ coef - y
    N_min = c1 * np.sum(np.abs(residuals_opt)) + c2 * np.sum(residuals_opt ** 2)
    
    # 2. Подынтегральная функция: exp( -(N_a(a) - N_min) )
    # В точке a=coef: N_a(a) - N_min = 0 → exp(0) = 1 (максимум подынтегральной функции)
    def integrand(*a):
        res = X @ np.asarray(a) - y
        # Пропускаем точки с экстремальными остатками (вклад ≈ 0)
        if np.any(np.abs(res) > 1e6):
            return 0.0
        N = c1 * np.sum(np.abs(res)) + c2 * np.sum(res ** 2)
        delta = N - N_min  # delta >= 0 (если coef — глобальный минимум в области)
        return np.exp(-delta) if delta < 700 else 0.0  # Защита от underflow
    
    # 3. Вычисляем I = ∫ exp( -(N_a(a) - N_min) ) da
    I, _ = nquad(integrand, ranges, opts={'epsabs': 1e-6, 'epsrel': 1e-4})
    
    if I <= 0:
        raise ValueError("Интеграл вычислен как неположительный. Проверьте param_range: он должен включать оценённые коэффициенты.")
    
    # 4. Собираем ln Z и BIC
    ln_Z = -N_min + np.log(I)
    BIC = k * np.log(n) + 2 * N_min + 2 * ln_Z  # N_a(coef) = N_min
    return BIC

In [ ]:
best_coefficients, best_y_pred, best_c1, best_c2 = select_best_mixed_norm(x, y, degree=2)
X = np.vander(x, N=2 + 1, increasing=True)
bic_entropy_stable(X, y, best_coefficients, 3, best_c1, best_c2)

KeyboardInterrupt: 

In [27]:
from scipy.integrate import quad
invexp = lambda x: np.exp(-x)
quad(invexp, 0, np.inf)

(1.0000000000000002, 5.842606703608969e-11)

# Оценка

## Метрики

### Интегральные

In [ ]:
def imse(g_hat, g_true, a=0, b=1, epsabs=1e-6):
    integrand = lambda x: (g_hat(x) - g_true(x)) ** 2
    integral, _ = quad(integrand, a, b, epsabs=epsabs, limit=100)
    return integral / (b - a)

def imae(g_hat, g_true, a=0, b=1, epsabs=1e-6):
    integrand = lambda x: abs(g_hat(x) - g_true(x))
    integral, _ = quad(integrand, a, b, epsabs=epsabs, limit=100)
    return integral / (b - a)

def maxerr(g_hat, g_true, a=0, b=1, xatol=1e-6):
    objective = lambda x: -abs(g_hat(x) - g_true(x))
    result = minimize_scalar(objective, bounds=(a, b), method='bounded', options={'xatol': xatol})
    if result.success:
        return -result.fun
    x_grid = np.linspace(a, b, 500)
    return max(abs(g_hat(x) - g_true(x)) for x in x_grid)


## Тестирование на полиномиальных данных

In [ ]:
log_df = pd.read_csv('../datasets/synthetic/synthetic_datasets_with_coeffs/seed_log.csv')
results = []

for degree in range(1, 7):
    for noise_level in ['low', 'moderate', 'high']:
        df = pd.read_csv(f'../datasets/synthetic/synthetic_datasets_with_coeffs/noise_{noise_level}_deg{degree}.csv')
        metrics = {'silverman': {'imse': [], 'imae': [], 'maxerr': []},
                   'cv': {'imse': [], 'imae': [], 'maxerr': []}}
        
        for seed in df['seed'].unique()[:30]:
            subset = df[df['seed'] == seed]
            x_train = subset['x'].values
            y_train = subset['y_noisy'].values
            
            # Истинная регрессия
            coeffs = [subset[f'coeff_{i}'].iloc[0] for i in range(degree + 1)]
            g_true = lambda x, c=coeffs: sum(c[i] * x**i for i in range(len(c)))
            
            # Модели
            model_map = kernel_regression_silverman(x_train, y_train)
            
            # Callable для метрик
            g_map = lambda x, m=model_silverman: m.fit(np.array([[x]]))[0][0]
            g_cv = lambda x, m=model_cv: m.fit(np.array([[x]]))[0][0]
            
            # Расчёт метрик (интегральные + дискретные для валидации)
            for method, g_hat in [('silverman', g_silverman), ('cv', g_cv)]:
                try:
                    metrics[method]['imse'].append(imse(g_hat, g_true))
                    metrics[method]['imae'].append(imae(g_hat, g_true))
                    metrics[method]['maxerr'].append(maxerr(g_hat, g_true))
                except:
                    # Фолбэк на дискретные метрики при сбое интегрирования
                    metrics[method]['imse'].append(imse_discrete(g_hat, g_true))
                    metrics[method]['imae'].append(imae_discrete(g_hat, g_true))
                    metrics[method]['maxerr'].append(maxerr_discrete(g_hat, g_true))
        
        # Агрегация результатов
        for method in ['silverman', 'cv']:
            m = metrics[method]
            results.append({
                'degree': degree,
                'noise_level': noise_level,
                'method': method,
                'imse_mean': np.mean(m['imse']),
                'imse_sem': np.std(m['imse']) / np.sqrt(len(m['imse'])),  # ← SEM вместо σ
                'imae_mean': np.mean(m['imae']),
                'imae_sem': np.std(m['imae']) / np.sqrt(len(m['imae'])),
                'maxerr_mean': np.mean(m['maxerr']),
                'maxerr_sem': np.std(m['maxerr']) / np.sqrt(len(m['maxerr']))
            })

# Сохранение и вывод
results_df = pd.DataFrame(results)
results_df.to_csv('kernel_regression_polynomial_results.csv', index=False)

print("Результаты ядерной регрессии на полиномиальных данных")
print("=" * 85)
print(results_df.to_string(index=False, float_format='%.6f'))
print("=" * 85)
print(f"\nСохранено: kernel_regression_polynomial_results.csv")